In [281]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

data = pd.read_csv('crime.csv')
df_or = pd.DataFrame(data)


#Some basic data cleaning here (ASK MUKESH ABOUT FURTHER DATA CLEANING)

#Remove empty data here (NaN)
df_or = df_or.dropna(subset = ['Population', 'City'])
df_or = df_or.drop('Unnamed: 13', axis=1)
df_or = df_or.drop(df_or.columns[4], axis = 1) #'Rape (revised definition)1'

#Remove commas from the numbers in all data
df_or = df_or.replace(',', '', regex=True)

#Renaming columns for consistency
df = df_or.rename({'Robbery':'robbery',
               df.columns[0]:'city',
               df.columns[1]:'population', 
               df.columns[2]:'violent crime',
               df.columns[3]:'murder',
               df.columns[4]:'rape',
               df.columns[6]:'assault',
               df.columns[7]:'property crime',
               df.columns[8]:'burglary',
               df.columns[9]:'larceny',
               df.columns[10]:'motor vehicle theft',
               df.columns[11]:'arson'}, 
               axis='columns')

#change murder and arson into integers instead of floats (remove decimals)
df['murder'] = df['murder'].astype(int)

#replace na values to 0 in column 'arson'
df['arson'] = df['arson'].apply(pd.to_numeric, errors='coerce').fillna(0, downcast='infer')


In [327]:
#converting all columns to integers instead of strings, this will allow us to use mathematical
#formulas on the data
df['population'] = df['population'].astype(int)
df['rape'] = df['rape'].astype(int)
df['assault'] = df['assault'].astype(int)
df['property crime'] = df['property crime'].astype(int)
df['burglary'] = df['burglary'].astype(int)
df['larceny'] = df['larceny'].astype(int)
df['motor vehicle theft'] = df['motor vehicle theft'].astype(int)
df['robbery'] = df['robbery'].astype(int)

#(ASK MUKESH) would this work        df = (df != df['city']).astype(int)
 
# As per instructions: Create new feature of population^2
df['population^2'] = np.power(df['population'], 2)

In [348]:
#as per instructions for robbery and murder: 
# "For this model, please use these variables to create categorical 
# features where values greater than 0 are coded 1, and values equal to 0 are coded 0."" 
df.murder[df.murder >= 1] = 1
df.murder[df.murder < 1] = 0

df.robbery[df.robbery >= 1] = 1
df.robbery[df.robbery < 1] = 0

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying 

In [349]:
df

,city,population,violent crime,murder,rape,robbery,assault,property crime,burglary,larceny,motor vehicle theft,arson,population^2
0,Adams Village,1861,0,0,0,0,0,12,2,10,0,0,3463321
1,Addison Town and Village,2577,3,0,0,0,3,24,3,20,1,0,6640929
2,Akron Village,2846,3,0,0,0,3,16,1,15,0,0,8099716
3,Albany,97956,791,1,30,1,526,4090,705,3243,142,0,9595377936
4,Albion Village,6388,23,0,3,1,16,223,53,165,5,0,40806544
5,Alfred Village,4089,5,0,0,1,2,46,10,36,0,0,16719921
6,Allegany Village,1781,3,0,0,0,3,10,0,10,0,0,3171961
7,Amherst Town,118296,107,1,7,1,68,2118,204,1882,32,3,13993943616
8,Amityville Village,9519,9,0,2,1,3,210,16,188,6,1,90611361
9,Amsterdam,18182,30,0,0,1,18,405,99,291,15,0,330585124
